In [ ]:
MAIN FEATURE

In [ ]:
As an introduction, this presentation does not follow our real code structure. 
In the beginning we prop the user for the youtube playlist he wishes to import to his spotify.

In [1]:
playlist_link = "https://www.youtube.com/playlist?list=PL0gdCDw52PLWC3Im-OP3CfyJFozvqCSuH"


The first step in the project is to open a browser page and scrap the provided youtube playlist using Selenium.
Our objective is to retrieve the songs titles of the playlist

In [2]:
from time import sleep
from selenium import webdriver
from tqdm import tqdm



def init(playlist_link):
    global num_of_videos
    global browser
    browser = webdriver.Chrome()
    browser.get(playlist_link)

    stats = browser.find_element_by_id("stats")
    num_of_videos = stats.text.split()[0]
    play_all_button = browser.find_element_by_xpath(
        '//*[@class="style-scope ytd-playlist-sidebar-primary-info-renderer"]')
    play_all_button.click()
    sleep(2)
    mute_button = browser.find_element_by_xpath(
        '//*[@class="ytp-mute-button ytp-button"]')
    mute_button.click()


def scrape_titles():
    titles = []
    for _ in tqdm(range(int(num_of_videos))):
        title = browser.find_element_by_xpath(
            '//*[@class="style-scope ytd-video-primary-info-renderer"]') \
            .text.splitlines()[0]        
        titles.append(title)
        next_button = browser.find_element_by_xpath(
            '//*[@class="ytp-next-button ytp-button"]')
        browser.execute_script("arguments[0].click();", next_button)
        #next_button.click()
        sleep(2) # If we can find a way to have this be more dynamic, 
                 # and not hard-coded to 2, that'd be nice
    browser.close()
    return titles



init(playlist_link)
titles = scrape_titles()

titles


100%|██████████| 15/15 [00:31<00:00,  2.10s/it]


['Camila Cabello - Havana ft. Young Thug',
 'The Chainsmokers & Coldplay - Something Just Like This (Lyric)',
 'ZAYN, Taylor Swift - I Don’t Wanna Live Forever (Fifty Shades Darker)',
 'The Chainsmokers - Closer ft. Halsey (Slushii Remix)',
 'The Weeknd - Starboy (official) ft. Daft Punk',
 'Marshmallo - iNvinCibLe',
 'marshmello - WaNt U 2 (marshmello x slushii Remix)',
 "The Weeknd - Can't Feel My Face",
 'The Weeknd - The Hills',
 'Ellie Goulding - Love Me Like You Do (Official Video)',
 'M83 - Intro',
 'Deadmau5 - Ghosts N Stuff - Nero Remix [FULL HD]',
 'The naked famous - Young blood ( The Art of Flight soundtrack)',
 'Camila Cabello - Havana Live (Latin American Music Awards 2017) Spanglish Version',
 'The Chainsmokers & Coldplay - Something Just Like This (Official Video HD) Legenda em português']

In order to attempt to add these songs into our spotify playlist, we need to use the Spotify API to search if the songs exist and retrieve their unique Spotify IDs necessary to add them to a playlist.
The format of the HTTP request to retrieve the IDs requires that the titles get converted to URL format

In [3]:
titles_as_URL = [title.replace(" ", "%20") for title in titles]

titles_as_URL

['Camila%20Cabello%20-%20Havana%20ft.%20Young%20Thug',
 'The%20Chainsmokers%20&%20Coldplay%20-%20Something%20Just%20Like%20This%20(Lyric)',
 'ZAYN,%20Taylor%20Swift%20-%20I%20Don’t%20Wanna%20Live%20Forever%20(Fifty%20Shades%20Darker)',
 'The%20Chainsmokers%20-%20Closer%20ft.%20Halsey%20(Slushii%20Remix)',
 'The%20Weeknd%20-%20Starboy%20(official)%20ft.%20Daft%20Punk',
 'Marshmallo%20-%20iNvinCibLe',
 'marshmello%20-%20WaNt%20U%202%20(marshmello%20x%20slushii%20Remix)',
 "The%20Weeknd%20-%20Can't%20Feel%20My%20Face",
 'The%20Weeknd%20-%20The%20Hills',
 'Ellie%20Goulding%20-%20Love%20Me%20Like%20You%20Do%20(Official%20Video)',
 'M83%20-%20Intro',
 'Deadmau5%20-%20Ghosts%20N%20Stuff%20-%20Nero%20Remix%20[FULL%20HD]',
 'The%20naked%20famous%20-%20Young%20blood%20(%20The%20Art%20of%20Flight%20soundtrack)',
 'Camila%20Cabello%20-%20Havana%20Live%20(Latin%20American%20Music%20Awards%202017)%20Spanglish%20Version',
 'The%20Chainsmokers%20&%20Coldplay%20-%20Something%20Just%20Like%20This%20(Off

Now we prop the user for additional information required to create a spotify playlist via the API (We detail in the Readme file of the git repo how to retrieve these information). The Token having a time validity, we ask for these information after the scrapping (if the playlist is big the Token could have expired if we retrieve it at the beginning of the process. 

In [4]:
spotify_username = "thom.thimothee"
spotify_api_token = "BQDraY62gneNBMrI9ThMNxdCxU_cFsKNF5zjjciqLunIf28RGHkchK7kFWot02BBSV3QndCBC-x7ih8jW8-RSDW6gODJtA_Pl3GsoVKY1cvVkUdIYzFq512v023XpM8pOh15Z9A5RfEBzqbIy1eJjJIToTtXvctscomKkS5u1g48ZzgLxW5fI2ehahwSW6tCE4xFo3xb5lwAFQbq3tyvwcS1CMDMU4McFTiLQJsLzBYc3h32Dx5ZHtTXwA"
playlist_name = "Youtube_Playlist"
playlist_description = "Created from Python script"

We now attend to create a playlist on the spotify account of the user. In the code, we have included some error handling in case some provided information were wrong. (Real code contain error handling)

In [5]:
import requests
import json

def create_playlist(ID, token, playlist_name, playlist_description):
    global user_id
    user_id = ID

    url_create_playlist = "https://api.spotify.com/v1/users/" + \
                          f"{user_id}/playlists"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    body = {
        "name": playlist_name,
        "description": playlist_description
    }
    r = requests.post(url_create_playlist, 
                      headers=headers,
                      data=json.dumps(body))
    results = r.json()
    
    return results["id"]

playlist_id = create_playlist(spotify_username, 
                                spotify_api_token,
                                playlist_name, 
                                playlist_description)
playlist_id

'3jz9laJtlvKVqIwob494VI'

We now try to search for our youtube songs

In [6]:
def find_spotify_songs(titles_as_URL, token):
    uris = []
    unknown_songs = []
    for url_title in titles_as_URL:
        url = f"https://api.spotify.com/v1/search?q={url_title}&type=track"
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {token}"
        }
        r = requests.get(url, headers=headers)
        results = r.json()
        try:
            uris.append(results["tracks"]["items"][0]["uri"])
        except KeyError:
            if results["error"]:
                print(results["error"]["message"])
        except IndexError:
            unknown_songs.append(url_title.replace("%20", " "))
    return (uris, unknown_songs)

uris, unknown_songs = find_spotify_songs(titles_as_URL, spotify_api_token)
uris

['spotify:track:1Zx5Jt8zJdxLgBmdQ2PqaP',
 'spotify:track:08hm0V6cTsWDX62hW42sEv',
 'spotify:track:3NdDpSvN911VPGivFlV5d0',
 'spotify:track:6RsWqX8zABZLhZydXxEFOm',
 'spotify:track:25khomWgBVamSdKw7hzm3l',
 'spotify:track:7w0FV6ViNDZFy9Mu90sQzl',
 'spotify:track:08hm0V6cTsWDX62hW42sEv']

In [7]:
unknown_songs

['The Chainsmokers - Closer ft. Halsey (Slushii Remix)',
 'The Weeknd - Starboy (official) ft. Daft Punk',
 'Marshmallo - iNvinCibLe',
 'marshmello - WaNt U 2 (marshmello x slushii Remix)',
 'Ellie Goulding - Love Me Like You Do (Official Video)',
 'Deadmau5 - Ghosts N Stuff - Nero Remix [FULL HD]',
 'The naked famous - Young blood ( The Art of Flight soundtrack)',
 'Camila Cabello - Havana Live (Latin American Music Awards 2017) Spanglish Version']

Finally we add the found songs into the created playlist

In [8]:
def add_to_playlist(uris, TOKEN, playlist_id):
    url = f"https://api.spotify.com/v1/users/{user_id}/playlists/" + \
          f"{playlist_id}/tracks"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {TOKEN}"
    }
    body = {
        "uris": uris
    }
    requests.post(url, headers=headers,
                       data=json.dumps(body))
                       
add_to_playlist(uris, spotify_api_token, playlist_id)

As an information to the user, we print the songs that were in the youtube playlist but not part of the Spotify song library

In [9]:
if len(unknown_songs) > 0:
    print("The following songs couldn't be added to the playlist:")
    for song in unknown_songs:
        print(song)

The following songs couldn't be added to the playlist:
The Chainsmokers - Closer ft. Halsey (Slushii Remix)
The Weeknd - Starboy (official) ft. Daft Punk
Marshmallo - iNvinCibLe
marshmello - WaNt U 2 (marshmello x slushii Remix)
Ellie Goulding - Love Me Like You Do (Official Video)
Deadmau5 - Ghosts N Stuff - Nero Remix [FULL HD]
The naked famous - Young blood ( The Art of Flight soundtrack)
Camila Cabello - Havana Live (Latin American Music Awards 2017) Spanglish Version
